## Step 1: Setting up the database

It took us a while to find a machine on our end that would work (issues with computer performance, computer architecture, server difficulties with getting container IPs), but our clinic computer was the ticket. The only complicating factor was that one of the Docker images wouldn’t build, so we built in on another machine, saved the built Docker image as a .tar file, loaded it onto our clinic computer, and loaded the .tar file into Docker on there. Aside from that, the setup was pretty straightforward!


## Step 2: Queries

### QUERY ONE: Join the ISD and TIGER tables together.
With this query, we need to process the LATITUDE and LONGITUDE columns in the ISD table as geography::Points, then check which Polygon in the TIGER GeographyLocation column that they lie within. We can do all this in a join to link the tables together:


In [ ]:
SELECT * FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY] as i
INNER JOIN [TIGERFiles].[dbo].[tl_2020_us_zcta] as t
ON t.GeographyLocation.STIntersects(geography::Point([LATITUDE], [LONGITUDE], 4326)) = 1
ORDER BY STATE asc;

The main problem with this is that it doesn’t complete in a reasonable time! This is because we aren’t using spatial indexing to the fullest here. But only joining the top 1000 entries in ISD and pulling off the distinct names and zips shows that things are looking good. We were able to improve performance by using a spatial index with maximum grid resolution (high at all four levels):

In [ ]:
CREATE SPATIAL INDEX WeatherLocationsInd ON [ISD_HMC].[2021_USA].[ISD_Spatial](GeographyLocationIsd)
WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH))


SELECT i.STATION_NAME, t.zcta5ce20
   FROM [ISD_HMC].[2021_USA].[ISD_Spatial] as i
       WITH (INDEX (WeatherLocationsInd))
   INNER JOIN [TIGERFiles].[dbo].[tl_2020_us_zcta] as t
   ON t.GeographyLocation.STIntersects(i.GeographyLocationIsd) = 1;

This query took 7 minutes and 32 seconds to complete on our clinic computer. It is likely that more could be done with spatial index options, but for now this is a promising result!

### QUERY TWO: Find the average annual temperature in Boston.


Because the ISD table contains a group of weather stations named ‘BOSTON MA’, the most obvious approach is to find the average temperature using those rows:


In [ ]:
SELECT NAME, AVG (TEMP) as "AVG TEMP" FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY] WHERE NAME = 'BOSTON MA' AND TEMP != 9999.9
GROUP BY NAME

This gives us an average temperature of 55.065 degrees. However, we weren’t confident that all of the weather stations in Boston were named ‘BOSTON MA’. We wrote some Python code that pulls the zip codes for Boston using a package, then prints out the SQL needed to make the temp table:

```
import zipcodes

tableName = "#BostonZipcodes"
desired_zips = zipcodes.filter_by(city="Boston")
zip_str = ""
for d in range(len(desired_zips)):
   zip_str = zip_str + " (" + desired_zips[d]['zip_code'] + "),"

print("CREATE TABLE " + tableName + " ( zip INT NOT NULL );")

print("INSERT INTO " + tableName + "(zip) VALUES " + zip_str[:-1] + ";")
```

In [ ]:
-- Create temporary table of Boston zipcodes
CREATE TABLE #BostonZipcodes (
   zip INT NOT NULL
);


-- Insert relevant zip codes
INSERT INTO #BostonZipcodes(zip)
VALUES (02108), (02109), (02110), (02111), (02112), (02113), (02114), (02115), (02116), (02117), (02118), (02119), (02120), (02121), (02122), (02123), (02124), (02125), (02126), (02127), (02128), (02129), (02130), (02131), (02132), (02133), (02134), (02135), (02136), (02137), (02141), (02149), (02150), (02151), (02152), (02163), (02171), (02196), (02199), (02201), (02203), (02204), (02205), (02206), (02210), (02211), (02212), (02215), (02217), (02222), (02241), (02266), (02283), (02284), (02293), (02297), (02298), (02445), (02467);

From there, we do a join similar to our work in query 1, but we do it on a subset of both tables (only the MA rows in ISD, only the rows from TIGER that have the zip codes from our temp table) so it actually runs!

In [ ]:
-- Extract rows from TIGER that have Boston zipcodes, 
SELECT i.NAME, AVG(i.TEMP) as "AVG_TEMP"
   FROM (
    SELECT *
       FROM [ISD_HMC].[2021_USA].[ISD_Spatial]
       WITH (INDEX (WeatherLocationsInd))
       WHERE STATE = 'MA'
    ) as i
INNER JOIN (
    SELECT *
       FROM [TIGERFiles].[dbo].[tl_2020_us_zcta], #BostonZipcodes as b
       WHERE zcta5ce20 = b.zip
   ) as t
ON t.GeographyLocation.STIntersects(i.GeographyLocationIsd) = 1
GROUP BY NAME


We get the same result as before, but doing it with the zip codes was a good check that there aren’t data validity issues in this case!
There was one more data validity concern in that one of the Boston weather stations did not report a temperature for every day of the year, throwing off our averages slightly.

The calculated average for temperature in Boston isn't far off from the true results of 52 degrees!

### QUERY THREE: What are the top five windiest stations in Massachusetts?

We decided that this could either mean highest average wind speed or highest maximum wind speed, so we did both:


In [ ]:
SELECT Top(5) MAX (WDSP) AS "MAX WIND SPEED", NAME FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY] WHERE [STATE] = 'MA' AND WDSP != 999.9
GROUP BY NAME
ORDER BY "MAX WIND SPEED" DESC;


SELECT Top(5) AVG (WDSP) AS "AVG WIND SPEED", NAME FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY] WHERE [STATE] = 'MA' AND WDSP != 999.9
GROUP BY NAME
ORDER BY "AVG WIND SPEED" DESC;

This query also could have benefitted from double-checking with the zip codes, but we opted to hold off on that until we’ve improved performance on the joins with spatial indexes.

It’s a little harder to find comparable data for this one, but our results here make sense, as all of these stations are either on Cape Cod, the islands, or relatively close to the seafront (in the case of Boston and Blue Hill).


### QUERY FOUR: Where is the rainiest station in Washington State located?

We could have just queried the ISD table for this one, but that would only get us position in terms of latitude and longitude. To get the zip code as well, we made a temp table then joined that data with TIGER:


In [ ]:
-- Make temp table that gets the total precipitation for each weather station in Washington State
SELECT SUM (PRCP) AS "TOTAL PRECIPITATION", NAME, LATITUDE, LONGITUDE
INTO #RainiestInWA
FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY]
WHERE [STATE] = 'WA' AND PRCP != 99.99
GROUP BY NAME, LATITUDE, LONGITUDE
ORDER BY "TOTAL PRECIPITATION" DESC;


-- Join temp table with zcta on Points in Polygons to get zip code column
SELECT TOP (1) "TOTAL PRECIPITATION", NAME, LATITUDE, LONGITUDE, t.zcta5ce20 as "ZIP CODE"
FROM #RainiestInWA
INNER JOIN [TIGERFiles].[dbo].[tl_2020_us_zcta] as t
ON t.GeographyLocation.STIntersects(geography::Point([LATITUDE], [LONGITUDE], 4326)) = 1
ORDER BY "TOTAL PRECIPITATION" DESC;

Like query 3, our analysis also would have benefitted from double-checking with the zip codes.

Quinault ranks highly on a list of average precipitation levels by town in Washington. All of the other top performers are in western WA (where Quinault is as well).


### QUERY FIVE: How many weather stations are there per state? Which state has the most weather stations? The least?

We tackled this question by counting all the distinct station names in each state in the ISD table:


In [ ]:
SELECT STATE, COUNT (DISTINCT NAME) AS "NUMBER OF STATIONS" FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY]
GROUP BY STATE
ORDER BY "NUMBER OF STATIONS" DESC;

The query gives us a top 20 that looks good...

In [ ]:
SELECT TOP(50) FROM
SELECT * FROM STATE, COUNT (DISTINCT NAME) AS "NUMBER OF STATIONS" FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY]
GROUP BY STATE
ORDER BY "NUMBER OF STATIONS" DESC;

…and a bottom 20 that looks a little strange:

In [ ]:
SELECT TOP(50) FROM
SELECT * FROM STATE, COUNT (DISTINCT NAME) AS "NUMBER OF STATIONS" FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY]
GROUP BY STATE
ORDER BY "NUMBER OF STATIONS" ASC;

We went through all of these unusual state codes using queries to see what happened with them. Some of the results were mislabeled, while others did not exist within the 50 states, or were weird all around:

| Unusual State Code | Actual State/Country                         |
|--------------------|----------------------------------------------|
| L,                 | Illinois (IL)                                |
| I,                 | Michigan (MI)                                |
| M,                 | Mostly oil platforms in the Gulf of Mexico   |
| Y,                 | New York (NY)                                |
| PR                 | Puerto Rico                                  |
| A,                 | 2 for Washington (WA), 1 for California (CA) |
| K,                 | Arkansas (AK)                                |
| D,                 | 1 for Maryland (MD), 1 for Mississippi (MS)  |
| X,                 | Texas (TX)                                   |
| VI                 | U.S. Virgin Islands                          |
| R,                 | Oregon (OR)                                  |
| RM                 | Marshall Islands                             |
| DC                 | Washington D.C.                              |
| C,                 | South Carolina (SC)                          |
| E,                 | Maine (ME)                                   |
| FM                 | Micronesia                                   |
| N,                 | Minnesota (MN)                               |

Repairing the faulty state codes gives Texas two more weather stations, so it has the most at 213. On the other hand, Delaware is the U.S. state with the fewest weather stations, having only 7 of them.

### QUERY SIX: Get the latitude and longitude for a station of your choosing.

For this one, we wanted to find the closest weather station to Harvey Mudd College. So we started by retrieving all the stations in California:


In [ ]:
SELECT DISTINCT NAME FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY] WHERE STATE = 'CA'

From there we found that the nearby Ontario International Airport was one of the included weather stations. So we opted for that one:

In [ ]:
SELECT DISTINCT NAME, LATITUDE, LONGITUDE FROM [ISD_HMC].[2021_USA].[DAILY_SUMMARY] WHERE NAME = 'ONTARIO INTERNATIONAL AIRPORT CA';

### QUERY SEVEN: Calculate the distance between the latitude and longitude values we choose in (6) and every station in the surrounding state (in both miles and meters)

We took the latitude and longitude from query 6, made it a fixed geography::Point in our query, then built distance columns using STDistance, the fixed point of Ontario, and all the other points using the LATITUDE and LONGITUDE columns. We converted our meters to miles by multiplying the meters result by the appropriate unit conversion:


In [ ]:
SELECT DISTINCT NAME, LATITUDE, LONGITUDE, geography::Point([LATITUDE], [LONGITUDE], 4326).STDistance(geography::Point(34.05314, -117.57689,4326)) AS 'DISTANCE (m)',
geography::Point([LATITUDE], [LONGITUDE], 4326).STDistance(geography::Point(34.05314, -117.57689,4326)) * 0.00062137 AS 'DISTANCE (mi)'
FROM  [ISD_HMC].[2021_USA].[DAILY_SUMMARY] WHERE STATE = 'CA'